 ### Домашнее задание по теме «Spark SQL»

Задача 1. Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [530]:
import findspark
findspark.init(r'C:\SPARK\spark-3.4.0-bin-hadoop3')
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [531]:
import pyspark
from pyspark.sql import SparkSession

In [533]:
spark = SparkSession.builder.getOrCreate()

In [534]:
df = spark.read.csv(r'C:\Users\user\Documents\IT\NETOLOGY\APACHE SPARK\owid-covid-data.csv')

In [535]:
type(df)

pyspark.sql.dataframe.DataFrame

In [536]:
df = spark.read.option('inferSchema', 'true').option('header', 'true').csv(r'C:\Users\user\Documents\IT\NETOLOGY\APACHE SPARK\owid-covid-data.csv')

In [537]:
from pyspark.sql.functions import col
from pyspark.sql.functions import dayofmonth
from pyspark.sql import functions as F
from pyspark.sql.functions import sum
from pyspark.sql.functions import round 
from pyspark.sql.functions import date_format
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
from pyspark.sql import Window as W
from pyspark.sql.functions import row_number
from pyspark.sql.functions import max
from pyspark.sql.functions import lag

In [ ]:
# не совсем понятно по какому срезу делать фильтр, так как дана просто дата
# поэтому я отфильтрую данные просто за 31 марта по всем годам, а потом просто суммирую

In [545]:
# сначала просто отфильтруем данные по месяцу марту и 31 числу
df_1 = df.select('iso_code', 'location', dayofmonth('date').alias('day_of_month'), 'total_cases', 'new_cases').where((date_format(col('date'), 'M')==3) & (dayofmonth('date')==31))

In [546]:
df_1.show(5)

+--------+-----------+------------+-----------+---------+
|iso_code|   location|day_of_month|total_cases|new_cases|
+--------+-----------+------------+-----------+---------+
|     AFG|Afghanistan|          31|      175.0|     29.0|
|     AFG|Afghanistan|          31|    56454.0|     70.0|
|OWID_AFR|     Africa|          31|     5781.0|    585.0|
|OWID_AFR|     Africa|          31|  4217313.0|  12752.0|
|     ALB|    Albania|          31|      243.0|     20.0|
+--------+-----------+------------+-----------+---------+
only showing top 5 rows



In [547]:
# не понятно, как считать количество переболевших. я количество переболевших считал как total_cases - new_cases
# процент переболевших считал как (total_cases - new_cases)/total_cases
df_1 = df_1.groupBy('iso_code', 'location').agg(sum('total_cases').alias('total_cases'), sum('new_cases').alias('new_cases'), (sum('total_cases')-sum('new_cases')).alias('recovered'),((sum('total_cases')-sum('new_cases'))/(sum('total_cases'))).alias('recovered_ratio'))

In [548]:
df_1.show(5)

+--------+-----------+-----------+---------+---------+------------------+
|iso_code|   location|total_cases|new_cases|recovered|   recovered_ratio|
+--------+-----------+-----------+---------+---------+------------------+
|     AUT|    Austria|   556409.0|   4249.0| 552160.0|0.9923635311434574|
|     COL|   Colombia|  2407283.0|   8754.0|2398529.0| 0.996363535155609|
|     BHS|    Bahamas|     9133.0|      0.0|   9133.0|               1.0|
|     CHN|      China|   184011.0|    103.0| 183908.0|0.9994402508545684|
|     AFG|Afghanistan|    56629.0|     99.0|  56530.0|0.9982517791237705|
+--------+-----------+-----------+---------+---------+------------------+
only showing top 5 rows



In [549]:
# вывоодим данные в порядке убывания и берем 15 строк
df_1 = df_1.select('iso_code', 'location', 'recovered_ratio').orderBy(F.col('recovered_ratio').desc()).show(15)

+--------+--------------------+---------------+
|iso_code|            location|recovered_ratio|
+--------+--------------------+---------------+
|     COM|             Comoros|            1.0|
|     ATG| Antigua and Barbuda|            1.0|
|     BHS|             Bahamas|            1.0|
|     GNQ|   Equatorial Guinea|            1.0|
|     HTI|               Haiti|            1.0|
|     FJI|                Fiji|            1.0|
|     COG|               Congo|            1.0|
|     CAF|Central African R...|            1.0|
|     GRD|             Grenada|            1.0|
|     FSM|Micronesia (country)|            1.0|
|     LBR|             Liberia|            1.0|
|OWID_INT|       International|            1.0|
|     MHL|    Marshall Islands|            1.0|
|     NER|               Niger|            1.0|
|     WSM|               Samoa|            1.0|
+--------+--------------------+---------------+
only showing top 15 rows



In [ ]:
# если имелось ввиду 31 марта 2021 года так как в остальных заданиях у нас фигурирует этот год
# то можно посчитать следующим образом

In [538]:
# фильтруем данные по дате 2021-03-31
df_2 = df.select('iso_code', 'location', 'date', 'total_cases', 'new_cases').where(df['date']=='2021-03-31')

In [539]:
# смотрим на данные
df_2.show()

+--------+-------------------+----------+-----------+---------+
|iso_code|           location|      date|total_cases|new_cases|
+--------+-------------------+----------+-----------+---------+
|     AFG|        Afghanistan|2021-03-31|    56454.0|     70.0|
|OWID_AFR|             Africa|2021-03-31|  4217313.0|  12752.0|
|     ALB|            Albania|2021-03-31|   125157.0|    434.0|
|     DZA|            Algeria|2021-03-31|   117192.0|    131.0|
|     AND|            Andorra|2021-03-31|    12010.0|     66.0|
|     AGO|             Angola|2021-03-31|    22311.0|    129.0|
|     AIA|           Anguilla|2021-03-31|       null|     null|
|     ATG|Antigua and Barbuda|2021-03-31|     1136.0|      0.0|
|     ARG|          Argentina|2021-03-31|  2348821.0|  16056.0|
|     ARM|            Armenia|2021-03-31|   192639.0|   1148.0|
|OWID_ASI|               Asia|2021-03-31|2.8507133E7| 183350.0|
|     AUS|          Australia|2021-03-31|    29322.0|     18.0|
|     AUT|            Austria|2021-03-31

In [555]:
# делаем группировку по 'iso_code' и 'location' и делаем все вычисления и затем сортируем в порядке убывания и выводим 15 строк
df_2.groupBy('iso_code', 'location').agg(((sum('total_cases')-sum('new_cases'))*100/sum('total_cases')).alias('ratio')).orderBy(F.col('ratio').desc()).show(15)

+--------+--------------------+-----+
|iso_code|            location|ratio|
+--------+--------------------+-----+
|     CAF|Central African R...|100.0|
|     TJK|          Tajikistan|100.0|
|     MHL|    Marshall Islands|100.0|
|     HTI|               Haiti|100.0|
|     LAO|                Laos|100.0|
|     LBR|             Liberia|100.0|
|     TGO|                Togo|100.0|
|     DMA|            Dominica|100.0|
|     WSM|               Samoa|100.0|
|     KNA|Saint Kitts and N...|100.0|
|     NIC|           Nicaragua|100.0|
|     NER|               Niger|100.0|
|     VAT|             Vatican|100.0|
|     COG|               Congo|100.0|
|     BWA|            Botswana|100.0|
+--------+--------------------+-----+
only showing top 15 rows



Задача 2. Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)

In [ ]:
# В колонке location помимо стран у нас содержатся также регионы и континенты
# Составим список чтобы потом в фильтре их исключить из выборки
excluded = ['Europe', 'European Union', 'South America', 'North America', 'World', 'Asia', 'Africa']

In [ ]:
# 1 способ решения с помощью оконных функций

In [563]:
# выделим год месяц и номер недели и искючим из выборки не страны
df_3 = df.select('date', date_format(col('date'), 'M').alias('Month'),date_format(col('date'), 'W').alias('week_num'), date_format(col('date'), 'Y').alias('Year'),date_format(col('date'), 'u').alias('week_day_num'), date_format(col('date'), 'E').alias('week_day_abb'), 'location', 'new_cases').where(~df.location.isin(excluded))

In [564]:
df_3.show(5)

+----------+-----+--------+----+------------+------------+-----------+---------+
|      date|Month|week_num|Year|week_day_num|week_day_abb|   location|new_cases|
+----------+-----+--------+----+------------+------------+-----------+---------+
|2020-02-24|    2|       5|2020|           1|         Mon|Afghanistan|      1.0|
|2020-02-25|    2|       5|2020|           2|         Tue|Afghanistan|      0.0|
|2020-02-26|    2|       5|2020|           3|         Wed|Afghanistan|      0.0|
|2020-02-27|    2|       5|2020|           4|         Thu|Afghanistan|      0.0|
|2020-02-28|    2|       5|2020|           5|         Fri|Afghanistan|      0.0|
+----------+-----+--------+----+------------+------------+-----------+---------+
only showing top 5 rows



In [565]:
# отфильтруем данные по последней неделе марта 2021 года
df_3 = df_3.select('date','week_num','week_day_num', 'week_day_abb', 'location', 'new_cases').where((df_3['Year']==2021) & (df_3['Month']==3) & (df_3['week_num']==5))

In [566]:
df_3.show(5)

+----------+--------+------------+------------+-----------+---------+
|      date|week_num|week_day_num|week_day_abb|   location|new_cases|
+----------+--------+------------+------------+-----------+---------+
|2021-03-28|       5|           7|         Sun|Afghanistan|      4.0|
|2021-03-29|       5|           1|         Mon|Afghanistan|     28.0|
|2021-03-30|       5|           2|         Tue|Afghanistan|     62.0|
|2021-03-31|       5|           3|         Wed|Afghanistan|     70.0|
|2021-03-28|       5|           7|         Sun|    Albania|    493.0|
+----------+--------+------------+------------+-----------+---------+
only showing top 5 rows



In [567]:
# c помощью оконной функции отсортируем данные по странам и с помощью row_number присвоим наибольшему количеству new_cases номер 1
df_3 = df_3.select('*', F.row_number().over(W.partitionBy('location').orderBy(col('new_cases').desc())).alias('row_num'))

In [568]:
df_3.show(5)

+----------+--------+------------+------------+-----------+---------+-------+
|      date|week_num|week_day_num|week_day_abb|   location|new_cases|row_num|
+----------+--------+------------+------------+-----------+---------+-------+
|2021-03-31|       5|           3|         Wed|Afghanistan|     70.0|      1|
|2021-03-30|       5|           2|         Tue|Afghanistan|     62.0|      2|
|2021-03-29|       5|           1|         Mon|Afghanistan|     28.0|      3|
|2021-03-28|       5|           7|         Sun|Afghanistan|      4.0|      4|
|2021-03-28|       5|           7|         Sun|    Albania|    493.0|      1|
+----------+--------+------------+------------+-----------+---------+-------+
only showing top 5 rows



In [570]:
# отфильтруем наши данные по row_number 1 и расположим количество new_cases в порядке убывания и выведем 15 строк
df_3.select('date', 'location', 'new_cases',).where(df_3['row_num']==1).orderBy(F.desc('new_cases')).show(15)

+----------+-------------+---------+
|      date|     location|new_cases|
+----------+-------------+---------+
|2021-03-31|       Brazil|  90638.0|
|2021-03-31|        India|  72330.0|
|2021-03-29|United States|  69429.0|
|2021-03-31|       France|  59054.0|
|2021-03-31|       Turkey|  39302.0|
|2021-03-31|       Poland|  32891.0|
|2021-03-31|      Germany|  25014.0|
|2021-03-31|        Italy|  23887.0|
|2021-03-30|       Sweden|  16427.0|
|2021-03-31|    Argentina|  16056.0|
|2021-03-31|         Peru|  15686.0|
|2021-03-29|        Spain|  15501.0|
|2021-03-28|      Ukraine|  12041.0|
|2021-03-31|         Iran|  10330.0|
|2021-03-29|  Philippines|  10002.0|
+----------+-------------+---------+
only showing top 15 rows



In [ ]:
# решим задачу 2 способом с помощью max и join

In [574]:
# отфильтруем наши данные по последней неделе марта и исключим не страны
df_4 = df.select('date', 'location', 'new_cases').where((date_format(col('date'), 'Y')==2021) & (date_format(col('date'), 'M')==3) & (date_format(col('date'), 'W')==5) & (~df.location.isin(excluded)))

In [575]:
df_4.show()

+----------+-----------+---------+
|      date|   location|new_cases|
+----------+-----------+---------+
|2021-03-28|Afghanistan|      4.0|
|2021-03-29|Afghanistan|     28.0|
|2021-03-30|Afghanistan|     62.0|
|2021-03-31|Afghanistan|     70.0|
|2021-03-28|    Albania|    493.0|
|2021-03-29|    Albania|    285.0|
|2021-03-30|    Albania|    304.0|
|2021-03-31|    Albania|    434.0|
|2021-03-28|    Algeria|     86.0|
|2021-03-29|    Algeria|    110.0|
|2021-03-30|    Algeria|    115.0|
|2021-03-31|    Algeria|    131.0|
|2021-03-28|    Andorra|     41.0|
|2021-03-29|    Andorra|     38.0|
|2021-03-30|    Andorra|     56.0|
|2021-03-31|    Andorra|     66.0|
|2021-03-28|     Angola|     32.0|
|2021-03-29|     Angola|     69.0|
|2021-03-30|     Angola|     50.0|
|2021-03-31|     Angola|    129.0|
+----------+-----------+---------+
only showing top 20 rows



In [577]:
# найдем максимальное значение new_cases для страны и расположим их в порядке убывания
df_5 = df_4.groupBy('location').agg(max('new_cases').alias('max_cases')).orderBy(F.desc(max('new_cases')))

In [578]:
df_5.show()

+-------------+---------+
|     location|max_cases|
+-------------+---------+
|       Brazil|  90638.0|
|        India|  72330.0|
|United States|  69429.0|
|       France|  59054.0|
|       Turkey|  39302.0|
|       Poland|  32891.0|
|      Germany|  25014.0|
|        Italy|  23887.0|
|       Sweden|  16427.0|
|    Argentina|  16056.0|
|         Peru|  15686.0|
|        Spain|  15501.0|
|      Ukraine|  12041.0|
|         Iran|  10330.0|
|  Philippines|  10002.0|
|       Serbia|   9983.0|
|      Hungary|   9082.0|
|       Russia|   8979.0|
|       Canada|   8728.0|
|      Czechia|   8664.0|
+-------------+---------+
only showing top 20 rows



In [579]:
# с помощью join соединим два датафрейма по location и max_cases, уберем ненужные колонки и отсортируем количество новых случаев в порядке убывания
# и выведем 15 строк. Проверяем первое и второе решение совпало.
df_4.join(df_5, (df_4["location"] == df_5["location"]) &
   ( df_4["new_cases"] == df_5["max_cases"])).drop(df_4['new_cases'], df_4['location']).orderBy(F.desc('max_cases')).show(15)

+----------+-------------+---------+
|      date|     location|max_cases|
+----------+-------------+---------+
|2021-03-31|       Brazil|  90638.0|
|2021-03-31|        India|  72330.0|
|2021-03-29|United States|  69429.0|
|2021-03-31|       France|  59054.0|
|2021-03-31|       Turkey|  39302.0|
|2021-03-31|       Poland|  32891.0|
|2021-03-31|      Germany|  25014.0|
|2021-03-31|        Italy|  23887.0|
|2021-03-30|       Sweden|  16427.0|
|2021-03-31|    Argentina|  16056.0|
|2021-03-31|         Peru|  15686.0|
|2021-03-29|        Spain|  15501.0|
|2021-03-28|      Ukraine|  12041.0|
|2021-03-31|         Iran|  10330.0|
|2021-03-29|  Philippines|  10002.0|
+----------+-------------+---------+
only showing top 15 rows



Задача 3. Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [580]:
# отфильтруем данные по последней неделе марта 2021 года и стране Россия
df_6 = df.select('date', date_format(col('date'), 'M').alias('Month'),date_format(col('date'), 'W').alias('week_num'), date_format(col('date'), 'Y').alias('Year'),date_format(col('date'), 'u').alias('week_day_num'), date_format(col('date'), 'E').alias('week_day_abb'), 'new_cases').where((date_format(col('date'), 'Y')==2021) & (date_format(col('date'), 'M')==3) & (date_format(col('date'), 'W')==5) & (F.col('location')=='Russia')).orderBy(F.col('date'))

In [581]:
df_6.show(5)

+----------+-----+--------+----+------------+------------+---------+
|      date|Month|week_num|Year|week_day_num|week_day_abb|new_cases|
+----------+-----+--------+----+------------+------------+---------+
|2021-03-28|    3|       5|2021|           7|         Sun|   8979.0|
|2021-03-29|    3|       5|2021|           1|         Mon|   8589.0|
|2021-03-30|    3|       5|2021|           2|         Tue|   8162.0|
|2021-03-31|    3|       5|2021|           3|         Wed|   8156.0|
+----------+-----+--------+----+------------+------------+---------+



In [582]:
# сгруппируем данные по дате и сумируем количество новых случаев
df_6 = df_6.groupBy('date').agg(sum('new_cases').alias('new_cases')).orderBy(F.col('date'))

In [584]:
df_6.show()

+----------+---------+
|      date|new_cases|
+----------+---------+
|2021-03-28|   8979.0|
|2021-03-29|   8589.0|
|2021-03-30|   8162.0|
|2021-03-31|   8156.0|
+----------+---------+



In [585]:
# с помощью оконной функции и метода lag найдем количество случаев на день ранее и посчитаем дельту
df_6.select('date', F.col('new_cases').alias('cases_today'), lag('new_cases', 1,).over(W.orderBy('date')).alias('cases_yesterday'), (F.col('new_cases') - lag('new_cases', 1,).over(W.orderBy('date'))).alias('delta')).show()

+----------+-----------+---------------+------+
|      date|cases_today|cases_yesterday| delta|
+----------+-----------+---------------+------+
|2021-03-28|     8979.0|           null|  null|
|2021-03-29|     8589.0|         8979.0|-390.0|
|2021-03-30|     8162.0|         8589.0|-427.0|
|2021-03-31|     8156.0|         8162.0|  -6.0|
+----------+-----------+---------------+------+



In [586]:
spark.stop()
sc.stop()